In [ ]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.182s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.147.60.144
npx: installed 22 in 2.9s
your url is: https://dark-islands-clean.loca.lt


In [ ]:
%%writefile app.py
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import scipy.stats as stats
from sklearn.model_selection import learning_curve
from imblearn.over_sampling import SMOTE
from collections import Counter
from tabulate import tabulate
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.metrics import log_loss

import streamlit as st

st.title("CREDIT CARD FRAUD DETECTION")
st.set_option('deprecation.showPyplotGlobalUse', False)
data=pd.read_csv("/content/drive/MyDrive/creditcard.csv")
data_=data.shape
tab1, tab2, tab3 = st.tabs(['Home','Visualization of data','Logistic Regression-Cross validation-Prediction for input from user'])

rbs = RobustScaler()
data_small = data[['Time','Amount']]
data_small = pd.DataFrame(rbs.fit_transform(data_small))
data_small.columns = ['scaled_time','scaled_amount']
data = pd.concat([data,data_small],axis=1)
data.drop(['Time','Amount'],axis=1,inplace=True)

non_fraud = data[data['Class']==0]
fraud = data[data['Class']==1]
new_data= pd.concat([non_fraud,fraud])
new_data = new_data.sample(frac=1)

target = 'Class'
predictors = ['scaled_time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',\
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',\
       'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',\
       'scaled_amount']
few_cols=['scaled_time', 'V1','V13','V16']

def metrics(model_name, y_train, y_test, y_hat_train, y_hat_test):
    '''Print out the evaluation metrics for a given models predictions'''

    st.write(f'Model: {model_name}', )
    st.write('-'*60)
    # Calculate confusion matrix
    cm_test = confusion_matrix(y_test, y_hat_test)
    cm_train = confusion_matrix(y_train, y_hat_train)

    #plot_confusion_matrix(y_test,y_hat_test)
    # Display confusion matrix for Test Data
    st.write('Confusion Matrix for Test Data:')
    printd=pd.DataFrame(cm_test, columns=["Predicted 0", "Predicted 1"], index=["Actual 0", "Actual 1"])
    st.write(printd)
    # Display confusion matrix as a plot
    plt.figure(figsize=(2, 1))
    sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    st.pyplot()
    # Display confusion matrix for Train Data
    st.write('\n\nConfusion Matrix for Train Data:')
    printd=pd.DataFrame(cm_train, columns=["Predicted 0", "Predicted 1"], index=["Actual 0", "Actual 1"])
    # Display confusion matrix for the training data as a plot
    plt.figure(figsize=(2, 1))
    sns.heatmap(cm_train, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    st.pyplot()
    st.write(f'Test accuracy: {round(accuracy_score(y_test, y_hat_test),2)}')
    st.write(f'Train accuracy: {round(accuracy_score(y_train, y_hat_train),2)}')
    st.write('-'*60)
    st.write('-'*60)
    st.write('Confusion Matrix:\n',pd.crosstab(y_test, y_hat_test,rownames=['Actual'],colnames=['Predicted'],margins = True))
    st.write('\nTest report:\n' + classification_report(y_test, y_hat_test))
    st.write('~'*60)
    st.write('\nTrain report:\n' + classification_report(y_train, y_hat_train))
    st.write('-'*60)
    st.write('\n\n')

def boxplot_(new_data):
    #outlier detection
    plt.style.use('dark_background')
    for var in few_cols:
        sns.boxplot(x=new_data[var],hue=new_data['Class'], palette='Set3')
        mean = new_data[var].mean()
        std = new_data[var].std()
        plt.axvline(mean - 3 * std, 0, 1)
        plt.text(mean - 3 * std, -0.55, 'mean - 3* std', rotation=60)
        plt.axvline(mean + 3 * std, 0, 1)
        plt.text(mean + 3 * std, -0.55, 'mean + 3* std', rotation=60)
        plt.show()
        st.pyplot()

def IQR_method(df,n,features):
    outlier_list = []

    for column in features:
        Q1 = np.percentile(df[column], 25)
        Q3 = np.percentile(df[column],75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR
        outlier_list_column = df[(df[column] < Q1 - outlier_step) | (df[column] > Q3 + outlier_step )].index
        outlier_list.extend(outlier_list_column)

    #selecting observations containing more than x outliers
    outlier_list = Counter(outlier_list)
    multiple_outliers = list( k for k, v in outlier_list.items() if v > n )

    out1 = df[df[column] < Q1 - outlier_step]
    out2 = df[df[column] > Q3 + outlier_step]

    print('Total number of deleted outliers is:', out1.shape[0]+out2.shape[0])

    return multiple_outliers

def pair_plot(new_data):
        plt.style.use('dark_background')
        sns.pairplot(new_data,hue='Class',vars=['scaled_time','scaled_amount'], palette='husl')
        st.pyplot()
def heatmap_(new_data):
        plt.style.use('dark_background')
        corr = new_data.corr()
        plt.figure(figsize=(25, 20))
        sns.heatmap(data=corr, annot=True, square=True, cbar=True)
        st.pyplot()
def histplot_(data):
        plt.figure(figsize=(10,6))
        sns.histplot(new_data['scaled_time'], bins=50, kde=True)
        plt.title('Distribution of Time')
        plt.show()
        st.pyplot()
def displot_(new_data):
        plt.style.use('dark_background')
        plt.figure(figsize=(10,6))
        sns.displot(new_data[new_data['Class'] == 1]['scaled_amount'], bins=50, label='Fraud', kde=False,color='green')
        plt.title('Distribution of Fraudulent Transactions')
        plt.xlabel('Amount')
        plt.ylabel('Number of Transactions')
        plt.show()
        st.pyplot()
def histplot__(new_data):
        p99 = new_data["scaled_amount"].quantile(0.99)
        sns.histplot(x="scaled_amount", hue="Class", bins=30,
             stat="probability", data=new_data[new_data["scaled_amount"] <= p99],
             common_norm=False);
        st.pyplot()
def scatter_plot(new_data):
        f,(ax1, ax2) = plt.subplots(2, 1, sharex=True)
        f.suptitle('Time of transaction vs Amount by class')
        ax1.scatter(fraud.scaled_time,fraud.scaled_amount)
        ax1.set_title('Fraud')
        ax2.scatter(non_fraud.scaled_time, non_fraud.scaled_amount)
        ax2.set_title('Non_fraud')
        plt.xlabel('Time (in Seconds)')
        plt.ylabel('Amount')
        plt.show();
        st.pyplot()
def check_normality(feature):
        plt.figure(figsize=(4, 4))
        ax1 = plt.subplot(1, 1, 1)
        stats.probplot(new_data[feature], dist=stats.norm, plot=ax1)
        ax1.set_title(f'{feature} Q-Q plot', fontsize=7)
        sns.despine()

        mean = new_data[feature].mean()
        std = new_data[feature].std()
        skew = new_data[feature].skew()
        st.write(f'{feature} : mean: {mean:.2f}, std: {std:.2f}, skew: {skew:.2f}')
        st.pyplot()

def time_density_plot(new_data):
        class_0 = new_data.loc[new_data['Class'] == 0]["scaled_time"]
        class_1 = new_data.loc[new_data['Class'] == 1]["scaled_time"]

        hist_data = [class_0, class_1]
        group_labels=["Non Fraud","Fraud"]
        fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
        fig['layout'].update(title='Credit Card Transactions Time Density Plot', xaxis=dict(title='Time [s]'))
        iplot(fig, filename='dist_only')
        st.pyplot()

def pie(new_data):
        new_data['Class'].value_counts()
        new_data['Class'].value_counts().plot.pie(explode=[0.1,0],autopct='%3.1f%%'
                                            , shadow=True, legend= True,startangle =45)
        plt.title('Distribution of Class',size=14)
        st.write('Genuine:', round(new_data['Class'].value_counts()[0]/len(new_data) * 100,2), '% of the dataset')
        st.write('Frauds:', round(new_data['Class'].value_counts()[1]/len(new_data) * 100,2), '% of the dataset')
        plt.show()
        st.pyplot()

def pie_plot(oversampled_data):
        oversampled_data['Class'].value_counts().plot.pie(explode=[0.1,0],autopct='%3.1f%%'
                                     ,shadow=True, legend= True,startangle =45)
        plt.title('Distribution of Class',size=14)
        plt.show()
        st.pyplot()

def feature_importance(tmp):
        # Sort the features by absolute coefficient value in descending order
        tmp['Abs_Coefficient'] = tmp['Coefficient'].abs()
        tmp = tmp.sort_values(by='Abs_Coefficient', ascending=False)

        plt.figure(figsize=(8,5))
        plt.title('Feature Importances', fontsize=14)
        s = sns.barplot(x='Feature', y='Abs_Coefficient', data=tmp)
        s.set_xticklabels(s.get_xticklabels(), rotation=90)
        plt.show()
        st.pyplot()

def cross_val(cvs):
        plt.plot(range(1, len(cvs) + 1), cvs, "o-")
        plt.title("Cross Val Score")
        plt.show()
        st.pyplot()

def partition(new_data):
        k = 5
        samples_per_partition = len(new_data) // k
        partitions = []
        # Split the dataset into k partitions with equal sample sizes
        start_index = 0
        for i in range(k):
            end_index = start_index + samples_per_partition
            partition = new_data.iloc[start_index:end_index]
            partitions.append(partition)
            start_index = end_index
        # Now, 'partitions' contains k DataFrames with equal sample sizes
        return partitions
def add_gaussian_noise(new_data, noise_std=0.1):
    noisy_df = new_data.copy()
    for col in new_data.columns:
        noisy_df[col] = new_data[col] + np.random.normal(0, noise_std, size=len(new_data))
    return noisy_df

with tab1:
    st.header('Fraud Detection')
    st.table(data.head(10))
    st.header('Credit Card Data description')
    df=data.describe()
    st.table(df)
    st.header('Details')
    st.subheader("Checking null values")
    temp=data.isnull().sum()
    st.table(temp)
    st.subheader('Shape of data')
    temp=data.shape
    st.subheader("Duplicated data")
    # Check for and count duplicated rows
    duplicated_count = data.duplicated().sum()
    st.write("Number of duplicated rows:", duplicated_count)
    st.write('Before processing',data_)
    st.write('After preprocessing',temp)
    data = new_data.drop_duplicates(keep='first')
    st.write("After preprocessing-Duplicated sum",data.duplicated().sum())
    total = new_data.isnull().sum().sort_values(ascending = False)
    percent = (new_data.isnull().sum()/new_data.isnull().count()*100).sort_values(ascending = False)
    pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).transpose()
    Outliers_IQR = IQR_method(new_data,1,predictors)
    st.subheader("Outlier's IQR")
    st.write(Outliers_IQR)

with tab2:
    new_data = new_data.drop_duplicates(keep='first')
    st.subheader("Boxplot for detecting outliers")
    boxplot_(new_data)
    st.header("Visualization of Data")
    st.subheader("Heatmap ")
    heatmap_(new_data)
    st.subheader("Pair-plot")
    pair_plot(new_data)
    st.subheader("Histogram for scaled time")
    histplot_(new_data)
    st.subheader("Displot ")
    displot_(new_data)
    st.subheader("Histplot for scaled amt")
    histplot__(new_data)
    st.subheader("Scatter plot")
    scatter_plot(new_data)
    st.subheader("Checking normality")
    check_normality("scaled_amount")
    st.subheader('Credit card transactions time density plot')
    time_density_plot(new_data)
    X = new_data.drop(columns='Class', axis=1)
    Y = new_data['Class']
    X.isnull().sum()
    total = X.isnull().sum().sort_values(ascending = False)
    percent = (X.isnull().sum()/X.isnull().count()*100).sort_values(ascending = False)
    st.write(pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).transpose())
    st.subheader("Oversampling using smote")
    st.write('Genuine:', round(new_data['Class'].value_counts()[0]/len(new_data) * 100,2), '% of the dataset')
    st.write('Frauds:', round(new_data['Class'].value_counts()[1]/len(new_data) * 100,2), '% of the dataset')
    pie(new_data)
    st.write(X.shape,Y.shape)
    smote = SMOTE(random_state=42)
    x_smote, y_smote = smote.fit_resample(X, Y)
    st.write(y_smote.value_counts())
    X_train, X_test, Y_train, Y_test = train_test_split(x_smote, y_smote,test_size=0.20, random_state=42)
    oversampled_data = pd.DataFrame(data=x_smote,columns=x_smote.columns)
    oversampled_data['Class']=y_smote
    oversampled_data.shape
    oversampled_data = oversampled_data.sample(frac=1)
    st.write(Counter(y_smote),oversampled_data.shape)
    st.write('Genuine:', round(oversampled_data['Class'].value_counts()[0]/len(oversampled_data) * 100,2), '% of the dataset')
    st.write('Frauds:', round(oversampled_data['Class'].value_counts()[1]/len(oversampled_data) * 100,2), '% of the dataset')
    st.subheader('Pie plot after oversampling ')
    pie_plot(oversampled_data)

with tab3:
    new_data = new_data.drop_duplicates(keep='first')
    smote = SMOTE(random_state=42)
    x_smote, y_smote = smote.fit_resample(X, Y)
    X_train, X_test, Y_train, Y_test = train_test_split(x_smote, y_smote,test_size=0.20, random_state=42)
    oversampled_data = pd.DataFrame(data=x_smote,columns=x_smote.columns)
    oversampled_data['Class']=y_smote
    oversampled_data.shape
    oversampled_data = oversampled_data.sample(frac=1)
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2)
    sc = StandardScaler()
    sc.fit(X_train)
    X_train= sc.transform(X_train)
    X_test = sc.transform(X_test)
    st.write('After standard scaling',X.shape, X_train.shape, X_test.shape)
    st.subheader('Checking for multicollinearity')
    #a DataFrame to hold VIF values
    vif_df = pd.DataFrame()
    vif_df['variable'] = X.columns

    #VIF for each predictor variable
    vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    st.write(vif_df)

    st.subheader('Removing multicollinearity with ridge regression')
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train, Y_train)
    variable_names = ["v1", "v2", "v3", "v4", "v5", "v6", "v7", "v8", "v9", "v10",
                    "v11", "v12", "v13", "v14", "v15", "v16", "v17", "v18", "v19", "v20",
                    "v21", "v22", "v23", "v24", "v25", "v26", "v27", "v28", "scaled_time","scaled_amount"]
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train, Y_train)

    # Coefficients after Ridge regularization
    coefficients = ridge.coef_
    for i, var in enumerate(variable_names):
        st.write(f'{var}: {coefficients[i]}')
    Y_train = Y_train.values.ravel()
    st.header("MODEL")

    st.subheader("Logistic Regression")
    model = LogisticRegression(random_state=42,max_iter=1000)
    model.fit(X_train, Y_train)

    st.subheader('Feature Importance')
    coefficients = model.coef_[0]

    # a DataFrame to store feature names and their corresponding coefficients
    tmp = pd.DataFrame({'Feature': predictors, 'Coefficient': coefficients})
    feature_importance(tmp)

    st.subheader('Model prediction')
    Y_pred = model.predict(X_train)
    training_data_accuracy = accuracy_score(Y_pred,Y_train)
    st.write('Accuracy on Training data : ', training_data_accuracy)
    X_test_prediction = model.predict(X_test)
    test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
    st.write('Accuracy score on Test Data : ', test_data_accuracy)

    models = []
    models.append(('LR', LogisticRegression(max_iter=1000)))

    for name, model in models:
        model.fit(X_train, Y_train)
        Y_hat_test = model.predict(X_test).astype(int)
        Y_hat_train = model.predict(X_train).astype(int)
        st.write(name, 'Accuracy Score is : ', accuracy_score(Y_test, Y_hat_test))
        st.write(Y_hat_test.shape,Y_hat_train.shape)
        st.write(Y_hat_test)
        metrics(name, Y_train, Y_test, Y_hat_train, Y_hat_test)

    st.subheader('Cross validation')
    cv = RepeatedKFold(n_splits=5, n_repeats= 100, random_state=1)
    model = LogisticRegression(max_iter=1000)

    scores = cross_validate(model, X_train, Y_train, scoring='accuracy', cv=5)
    st.write(scores['test_score'])
    st.write('Mean=',np.mean(scores['test_score']))
    cvs = cross_val_score(model,X,Y, cv=5, scoring="f1_macro")

    model.fit(X_train, Y_train)
    transaction_data = pd.DataFrame({
    'Time': [16],
    'V1': [1.322707269],
    'V2': [-0.174040833],
    'V3': [0.434555031],
    'V4': [0.576037652],
    'V5': [-0.836758046],
    'V6': [-0.831083411],
    'V7': [-0.264904961],
    'V8': [-0.220981943],
    'V9': [-1.071424618],
    'V10': [0.868558548],
    'V11': [-0.64150629],
    'V12': [-0.111315775],
    'V13': [0.36148541],
    'V14': [0.171945122],
    'V15': [0.782166532],
    'V16': [-1.35587073],
    'V17': [-0.216935153],
    'V18': [1.271765385],
    'V19': [-1.240621935],
    'V20': [-0.522950941],
    'V21': [-0.284375572],
    'V22': [-0.323357411],
    'V23': [-0.037709905],
    'V24': [0.347150939],
    'V25': [0.559639137],
    'V26': [-0.280158166],
    'V27': [0.042335258],
    'V28': [0.0288223],
    'Amount': [0],
    'Class': [0]
  })

    new_predictions = model.predict(X)

    if new_predictions[0] == 1:
        st.write("Time: 16\nPredict: Fraudulent Transaction")
    else:
        st.write("Time: 16\nPredict: Genuine Transaction")

    st.subheader('PATE')
    st.write("Training : ",X_train.shape,Y_train.shape)
    st.write("Testing : ",X_test.shape,Y_test.shape)

    def private_aggregate_teacher_predictions(oversampled_data, num_teachers, epsilon):
        x = oversampled_data.drop(columns=['Class'])
        y = oversampled_data['Class']

        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

        teacher_models = []

        for i in range(num_teachers):
            x_train_teacher, x_test_teacher, y_train_teacher, y_test_teacher = train_test_split(x_train, y_train, test_size=0.5, random_state=i)

            teacher_model = LogisticRegression(max_iter=1000)
            teacher_model.fit(x_train_teacher, y_train_teacher)
            teacher_models.append(teacher_model)

            teacher_accuracy = accuracy_score(y_test_teacher,teacher_model.predict(x_test_teacher))
            st.write(f"Teacher {i} accuracy: {teacher_accuracy}")

        teacher_predictions = []

        for teacher_model in teacher_models:
            teacher_pred = teacher_model.predict(x_test)
            teacher_predictions.append(teacher_pred)

        final_aggregated_predictions = []

        for i in range(len(x_test)):
            class_counts = [0, 0]

            for teacher_prediction in teacher_predictions:
                class_counts[teacher_prediction[i]] += 1

            #adding noise
            noisy_counts = [count + np.random.laplace(scale=1/epsilon) for count in class_counts]

            max_votes = max(noisy_counts)
            selected_classes = [j for j, vote in enumerate(noisy_counts) if vote == max_votes]
            final_prediction = np.random.choice(selected_classes)

            final_aggregated_predictions.append(final_prediction)

        aggregated_accuracy = accuracy_score(y_test,final_aggregated_predictions)
        st.write(f"Aggregated predictions accuracy: {aggregated_accuracy}")

        return final_aggregated_predictions

    num_teachers = 5
    epsilon = 1.0
    aggregated_predictions = private_aggregate_teacher_predictions(oversampled_data, num_teachers, epsilon)
    st.write("Final aggregated predictions for all test data points:", aggregated_predictions)

    st.subheader('Federated Learning')
    def sigmoid(z):
        return 1 / (1 + np.exp(-np.clip(z, -15, 15)))

    x = oversampled_data.drop(columns=['Class'])
    y = oversampled_data['Class']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    sample_fraction = 0.5
    x_train = x_train.sample(frac=sample_fraction, random_state=42)
    y_train = y_train.loc[x_train.index]

    num_clients = 2

    global_weights = np.zeros(x_train.shape[1])

    client_weights = [np.zeros(x_train.shape[1]) for _ in range(num_clients)]

    num_epochs = 1
    learning_rate = 0.1

    for epoch in range(num_epochs):
        for client_id in range(num_clients):
            local_weights = client_weights[client_id]
            for i in range(len(x_train)):
                x_i = x_train.iloc[i].values
                y_i = y_train.iloc[i]
                z = np.dot(x_i, local_weights)
                h = sigmoid(z)
                gradient = (h-y_i) * x_i
                local_weights -= learning_rate * gradient

            # Print log loss for each client after the first epoch
            if epoch == 0 or epoch==1:
                z_values = np.dot(x_train.values, local_weights)
                h_values = sigmoid(z_values)
                loss = log_loss(y_train,h_values)
                accuracy = accuracy_score(y_train,(h_values >= 0.5).astype(int))
                st.write(f'Client {client_id} - Log Loss: {loss}, Accuracy: {accuracy}')

            client_weights[client_id] = local_weights

        global_weights = np.mean(client_weights, axis=0)

    z_values = np.dot(x_test.values,global_weights)
    h_values = sigmoid(z_values)

    global_predictions = (h_values >= 0.5).astype(int)
    st.write("Global Predictions : ",global_predictions)

    global_accuracy = accuracy_score(y_test, global_predictions)
    st.write("Global Model Accuracy: ", global_accuracy)

Overwriting app.py
